In [1]:
from objects import EnvironmentManager
from params import *
from helpers import get_action
from utils import create_field_from_list_column
import numpy as np
import pandas as pd

In [2]:
env = EnvironmentManager(reward_weight=TRIP_WEIGHT)
env.start_vehicles()
env.route.load_all_pax(demand_scenario='off_peak')

observation, reward, done, terminated, info = env.step(action=None)
while not terminated:
    if not done:
        action = get_action(
            'DRD', observation, 
            minimum_requests_slope=1.0, base_minimum_requests=1.5) # ideal slope
    else:
        action = None
    observation, reward, done, terminated, info = env.step(action=action)
history = env.get_history()

In [3]:
pax = history['pax']
pax['arrival_hour'] = (pax['arrival_time']/3600).astype(int)
print(pax.groupby('flex').size())
# pax.groupby(['direction', 'origin', 'arrival_hour']).size().reset_index(name='count')

flex
0    355
1    115
dtype: int64


In [4]:
84/437

0.19221967963386727

In [5]:
110/437

0.2517162471395881

In [7]:
from eval import evaluate_heuristic

# function to receive a float and return a string with '_' instead of '.'
def float_to_str(f):
    return str(f).replace('.', '_')

main_history = {
    'pax': [],
    'vehicles': [],
    'idle': [],
}
N_EPISODES = 30
RANDOM_SEED = 1

# slope scenarios
alphas = [1.0, 1.2, 1.5, 1.8]
# alphas = [1.2,]

main_history = {
    'pax': [],
    # 'state': [],
    'vehicles': [],
    'idle': [],
}
N_EPISODES = 30
RANDOM_SEED = 1

for demand in ('peak', 'off_peak'):
    # heuristic
    for i in range(len(alphas)):
        scenario_name = f'heuristic_{float_to_str(alphas[i])}'
        np.random.seed(RANDOM_SEED)
        tmp_history = evaluate_heuristic(
            alpha=alphas[i], beta=HEURISTIC_BETA, n_episodes=N_EPISODES, 
            output_history=True,
            scenario_name=scenario_name, demand_scenario=demand)
        # for ky in tmp_history:
        #     if ky not in main_history:
        #         main_history[ky] = []
        #     tmp_history[ky]['demand'] = demand
        #     main_history[ky].append(tmp_history[ky])

Vehicle 1 at stop 1 with expected headway 91
Time difference: 59
Current time: 10887, Old depart time 10886.8 New depart time: 10945.6
Vehicle 0 at stop 3 with expected headway 83
Time difference: 67
Current time: 4974, Old depart time 4974.0 New depart time: 5041.2
Vehicle 0 at stop 0 with expected headway 124
Time difference: 26
Current time: 6031, Old depart time 6043.100000000002 New depart time: 6068.999999999999
Vehicle 0 at stop 1 with expected headway 116
Time difference: 34
Current time: 6165, Old depart time 6177.099999999999 New depart time: 6211.299999999999
Vehicle 0 at stop 3 with expected headway 80
Time difference: 70
Current time: 6271, Old depart time 6271.299999999999 New depart time: 6341.299999999999
Vehicle 0 at stop 1 with expected headway 142
Time difference: 8
Current time: 6682, Old depart time 6700.5 New depart time: 6708.2
Vehicle 0 at stop 5 with expected headway 146
Time difference: 4
Current time: 8218, Old depart time 8238.400000000001 New depart time: 8

In [46]:
env.route.vehicles[0].tracker['deviations']

15

In [45]:
env.route.vehicles[0].tracker['deviation_opportunities']

29

In [ ]:
history['state'].head(1)

KeyError: 'state'

In [42]:
for ky in env.state_hist[1]:
    print(ky, len(env.state_hist[1][ky]))

time 45
observation 45
action 46
reward 45
unweighted_rewards 45
next_observation 45
done 45


In [18]:
state = env.state_hist[1]
state['action'].append(np.nan)
for i in range(3):
    state['reward'].append(np.nan)
    state['done'].append(np.nan)
    state['unweighted_rewards'].append(np.nan)
    state['next_observation'].append(np.nan)   
state_df = pd.DataFrame(state)

In [19]:
state_df

,time,observation,action,reward,unweighted_rewards,next_observation,done
0,762.0,"[0, 1, 599.0, 0.0]",0.0,-5.0,"[1, 1]","[1, 1, 614.0, -43.0]",0.0
1,869.0,"[1, 1, 614.0, -43.0]",1.0,0.0,"[0, 0, 1]","[1, 0, 678.0, -1.0]",0.0
2,1309.0,"[0, 1, 567.0, -53.0]",1.0,0.0,"[0, 0, 1]","[1, 2, 499.0, -6.0]",0.0
3,1515.0,"[1, 0, 678.0, -1.0]",0.0,-4.0,"[0, 1]","[2, 0, 660.0, -7.0]",1.0
4,1930.0,"[0, 2, 589.0, -32.0]",1.0,-5.0,"[1, 1]","[1, 1, 605.0, 12.0]",0.0
5,2106.0,"[1, 2, 499.0, -6.0]",1.0,-5.0,"[1, 1]","[2, 0, 567.0, -17.0]",1.0
6,2559.0,"[0, 1, 598.0, 1.0]",0.0,-4.0,"[0, 1]","[1, 4, 685.0, 19.0]",0.0
7,2722.0,"[1, 1, 605.0, 12.0]",0.0,-4.0,"[0, 1]","[2, 0, 476.0, 21.0]",1.0
8,3133.0,"[0, 3, 573.0, -27.0]",1.0,0.0,"[0, 0]","[1, 0, 687.0, 104.0]",0.0
9,3327.0,"[1, 4, 685.0, 19.0]",1.0,0.0,"[0, 0]","[2, 0, 626.0, 63.0]",1.0


In [11]:
df

,time,observation
0,762.0,"[0, 1, 599.0, 0.0]"
1,869.0,"[1, 1, 614.0, -43.0]"
2,1309.0,"[0, 1, 567.0, -53.0]"
3,1515.0,"[1, 0, 678.0, -1.0]"
4,1930.0,"[0, 2, 589.0, -32.0]"
5,2106.0,"[1, 2, 499.0, -6.0]"
6,2559.0,"[0, 1, 598.0, 1.0]"
7,2722.0,"[1, 1, 605.0, 12.0]"
8,3133.0,"[0, 3, 573.0, -27.0]"
9,3327.0,"[1, 4, 685.0, 19.0]"


In [9]:
pd.DataFrame(env.state_hist[1]['time'])

,0
0,762.0
1,869.0
2,1309.0
3,1515.0
4,1930.0
5,2106.0
6,2559.0
7,2722.0
8,3133.0
9,3327.0


In [13]:
env.state_hist[1]['next_observation']

[[1, 1, 614.0, -43.0],
 [1, 0, 678.0, -1.0],
 [1, 2, 499.0, -6.0],
 [2, 0, 660.0, -7.0],
 [1, 1, 605.0, 12.0],
 [2, 0, 567.0, -17.0],
 [1, 4, 685.0, 19.0],
 [2, 0, 476.0, 21.0],
 [1, 0, 687.0, 104.0],
 [2, 0, 626.0, 63.0],
 [1, 0, 621.0, -50.0],
 [2, 0, 414.0, -29.0],
 [1, 0, 700.0, 33.0],
 [2, 0, 461.0, -13.0],
 [1, 0, 569.0, -95.0],
 [2, 0, 570.0, -126.0],
 [1, 0, 627.0, 36.0],
 [2, 0, 619.0, 11.0],
 [1, 1, 575.0, 11.0],
 [2, 0, 761.0, -51.0],
 [1, 1, 594.0, -109.0],
 [2, 0, 370.0, -80.0],
 [1, 0, 625.0, -20.0],
 [2, 0, 639.0, -78.0],
 [1, 0, 597.0, -31.0],
 [2, 0, 639.0, -44.0],
 [1, 1, 654.0, -11.0],
 [2, 0, 572.0, -22.0],
 [1, 0, 630.0, -16.0],
 [2, 0, 608.0, -62.0],
 [1, 1, 556.0, -15.0],
 [2, 0, 614.0, 31.0],
 [1, 1, 505.0, -48.0],
 [2, 0, 723.0, -49.0],
 [1, 2, 523.0, -17.0],
 [2, 0, 635.0, -6.0],
 [1, 0, 536.0, -111.0],
 [2, 0, 547.0, -130.0],
 [1, 0, 664.0, 46.0],
 [2, 0, 499.0, 49.0],
 [1, 0, 561.0, -26.0],
 [2, 0, 759.0, 39.0],
 [1, 0, 572.0, -32.0]]

In [12]:
env.state_hist[1]['observation']

[[0, 1, 599.0, 0.0],
 [1, 1, 614.0, -43.0],
 [0, 1, 567.0, -53.0],
 [1, 0, 678.0, -1.0],
 [0, 2, 589.0, -32.0],
 [1, 2, 499.0, -6.0],
 [0, 1, 598.0, 1.0],
 [1, 1, 605.0, 12.0],
 [0, 3, 573.0, -27.0],
 [1, 4, 685.0, 19.0],
 [0, 4, 661.0, 60.0],
 [1, 0, 687.0, 104.0],
 [0, 0, 685.0, 13.0],
 [1, 0, 621.0, -50.0],
 [0, 2, 643.0, 6.0],
 [1, 0, 700.0, 33.0],
 [0, 0, 629.0, -58.0],
 [1, 0, 569.0, -95.0],
 [0, 3, 611.0, -22.0],
 [1, 0, 627.0, 36.0],
 [0, 2, 572.0, -41.0],
 [1, 1, 575.0, 11.0],
 [0, 0, 587.0, -59.0],
 [1, 1, 594.0, -109.0],
 [0, 2, 598.0, -44.0],
 [1, 0, 625.0, -20.0],
 [0, 1, 586.0, -35.0],
 [1, 0, 597.0, -31.0],
 [0, 0, 626.0, -23.0],
 [1, 1, 654.0, -11.0],
 [0, 1, 570.0, -52.0],
 [1, 0, 630.0, -16.0],
 [0, 1, 598.0, -42.0],
 [1, 1, 556.0, -15.0],
 [0, 1, 616.0, 0.0],
 [1, 1, 505.0, -48.0],
 [0, 3, 543.0, -16.0],
 [1, 2, 523.0, -17.0],
 [0, 0, 576.0, -35.0],
 [1, 0, 536.0, -111.0],
 [0, 3, 629.0, -22.0],
 [1, 0, 664.0, 46.0],
 [0, 2, 596.0, -31.0],
 [1, 0, 561.0, -26.0],
 [0,

In [11]:
env.state_hist[1]['time']

[762.0,
 869.0,
 1309.0,
 1515.0,
 1930.0,
 2106.0,
 2559.0,
 2722.0,
 3133.0,
 3327.0,
 3822.0,
 4014.0,
 4373.0,
 4462.0,
 4964.0,
 5149.0,
 5518.0,
 5619.0,
 6142.0,
 6354.0,
 6719.0,
 6921.0,
 7291.0,
 7399.0,
 7916.0,
 8090.0,
 8527.0,
 8681.0,
 9139.0,
 9301.0,
 9714.0,
 9900.0,
 10316.0,
 10493.0,
 10962.0,
 11062.0,
 11550.0,
 11695.0,
 12127.0,
 12203.0,
 12742.0,
 12956.0,
 13331.0,
 13488.0,
 13884.0,
 14082.0]

In [4]:
histories = env.get_history()

ValueError: All arrays must be of the same length

In [12]:
import pandas as pd

In [14]:
pd.DataFrame(env.route.vehicles[0].event_hist).head(10)

,direction,trip_id,stop,arrival_time,departure_time,load,boardings,alightings,scheduled_time
0,out,0,0,0.0,0.0,0,0,0,0
1,out,0,1,122.0,122.0,0,0,0,150
2,out,0,3,201.0,201.0,0,0,0,300
3,out,0,5,287.0,295.0,1,1,0,450
4,out,0,6,422.0,430.0,0,0,1,600
5,in,0,0,600.0,600.0,0,0,0,600
6,in,0,1,757.0,757.0,0,0,0,750
7,in,0,3,925.0,925.0,0,0,0,900
8,in,0,5,1008.0,1008.0,0,0,0,1050
9,in,0,6,1134.0,1134.0,0,0,0,1200


In [8]:
env.state_hist[0]['unweighted_rewards']

[[0, 1],
 [1, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [1, 2, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [1, 0, 0],
 [2, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 1, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 1, 1],
 [0, 0, 1],
 [0, 0, 0],
 [0, 0, 1],
 [0, 0, 1]]

In [6]:
env.state_hist[0]['time']

[1342.0,
 1513.0,
 1930.0,
 2109.0,
 2564.0,
 2667.0,
 3150.0,
 3380.0,
 3831.0,
 3941.0,
 4309.0,
 4458.0,
 4889.0,
 5044.0,
 5532.0,
 5857.0,
 6214.0,
 6400.0,
 6783.0,
 6882.0,
 7336.0,
 7562.0,
 8071.0,
 8216.0,
 8577.0,
 8730.0,
 9090.0,
 9218.0,
 9705.0,
 9869.0,
 10313.0,
 10464.0,
 10985.0,
 11191.0,
 11595.0,
 11689.0,
 12096.0,
 12299.0,
 12727.0,
 12816.0,
 13299.0,
 13463.0,
 13961.0,
 14070.0]

In [5]:
env.state_hist[0]['observation']

[[0, 1, 596.0, -18.0],
 [1, 1, 539.0, 1.0],
 [0, 3, 632.0, -34.0],
 [1, 2, 718.0, -5.0],
 [0, 0, 652.0, 2.0],
 [1, 2, 642.0, -49.0],
 [0, 1, 609.0, -14.0],
 [1, 1, 689.0, 68.0],
 [0, 2, 607.0, 57.0],
 [1, 0, 587.0, 29.0],
 [0, 0, 578.0, -51.0],
 [1, 0, 535.0, -52.0],
 [0, 2, 581.0, -71.0],
 [1, 1, 612.0, -68.0],
 [0, 1, 563.0, -36.0],
 [1, 1, 736.0, 137.0],
 [0, 0, 615.0, 52.0],
 [1, 2, 723.0, 88.0],
 [0, 0, 663.0, 21.0],
 [1, 0, 561.0, -38.0],
 [0, 4, 569.0, -28.0],
 [1, 0, 703.0, 46.0],
 [0, 4, 796.0, 113.0],
 [1, 0, 760.0, 106.0],
 [0, 1, 657.0, 17.0],
 [1, 0, 632.0, 16.0],
 [0, 1, 509.0, -70.0],
 [1, 1, 506.0, -82.0],
 [0, 2, 549.0, -61.0],
 [1, 1, 530.0, -45.0],
 [0, 0, 557.0, -47.0],
 [1, 2, 516.0, -46.0],
 [0, 2, 619.0, 19.0],
 [1, 0, 643.0, 77.0],
 [0, 1, 624.0, 33.0],
 [1, 1, 536.0, -23.0],
 [0, 1, 592.0, -72.0],
 [1, 2, 663.0, -17.0],
 [0, 0, 593.0, -33.0],
 [1, 1, 508.0, -96.0],
 [0, 1, 563.0, -59.0],
 [1, 1, 502.0, -47.0],
 [0, 0, 631.0, 11.0],
 [1, 0, 541.0, -46.0]]

In [3]:
vehs = episode_history['vehicles'].copy()
vehs['arrival_delay'] = (vehs['arrival_time']-vehs['scheduled_time']).astype(int)
vehs['departure_delay'] = (vehs['departure_time']-vehs['scheduled_time']).astype(int)
vehs = vehs.drop(columns=['scheduled_time', 'load', 'boardings', 'alightings'])
state = episode_history['state'].copy()

In [4]:
# create_field_from_list_column(state, 3, 'delay', 'observation')
# create_field_from_list_column(state, 0, 'control_stop', 'observation')
# create_field_from_list_column(state, 1, 'delay_reward', 'unweighted_rewards')
# state['control_stop'] = state['control_stop'].astype(int)
# state['delay'] = state['delay'].astype(int)
# state['delay_reward'] = state['delay_reward'].astype(int)

# state['stop'] = state['control_stop'].map(CONTROL_STOPS_STOP_MAPPING)
# state['direction'] = state['control_stop'].map(CONTROL_STOPS_DIRECTION_MAPPING)

# violations = state[(state['delay_reward']==1) & (state['delay'].between(*ON_TIME_BOUNDS))].copy()
# violations = violations.drop(columns=['reward', 'unweighted_rewards', 'delay_reward','control_stop'])

# vehs_for_merge = vehs[['veh_idx', 'stop', 'direction', 'arrival_time', 'departure_time', 'arrival_delay', 'departure_delay']].copy()

# violations = violations.merge(vehs_for_merge, 
#                               how='left', 
#                               left_on=['veh_idx','stop', 'direction', 'time'], 
#                               right_on=['veh_idx','stop', 'direction', 'departure_time'])

In [5]:
state.head(20)

,time,observation,action,reward,unweighted_rewards,veh_idx
0,1342.0,"[0, 1, 596.0, -18.0]",1,0.0,"[0, 0]",0
1,1513.0,"[1, 1, 539.0, 1.0]",0,-5.0,"[1, 1]",0
2,1930.0,"[2, 3, 632.0, -34.0]",1,0.0,"[0, 0]",0
3,2109.0,"[3, 2, 718.0, -5.0]",1,0.0,"[0, 0]",0
4,2564.0,"[0, 0, 652.0, 2.0]",0,0.0,"[0, 0]",0
5,2667.0,"[1, 2, 642.0, -49.0]",1,0.0,"[0, 0]",0
6,3150.0,"[2, 1, 609.0, -14.0]",1,-4.0,"[0, 1]",0
7,3380.0,"[3, 1, 689.0, 68.0]",0,-5.0,"[1, 1]",0
8,3831.0,"[0, 2, 607.0, 57.0]",0,-2.0,"[2, 0]",0
9,3941.0,"[1, 0, 587.0, 29.0]",0,0.0,"[0, 0]",0


In [7]:
vehs[(vehs['arrival_time'].between(900, 2500)) & (vehs['veh_idx']==0)]

,direction,trip_id,stop,arrival_time,departure_time,veh_idx,headway,flex,arrival_delay,departure_delay
8,in,0,5,906.0,906.0,0,NaN,0,-144,-144
9,in,0,6,1014.0,1022.0,0,NaN,0,-186,-178
10,out,2,0,1200.0,1208.0,0,600.0,0,0,8
11,out,2,1,1306.0,1314.0,0,621.0,0,-44,-36
12,out,2,3,1426.0,1436.0,0,604.0,0,-74,-64
13,out,2,4,1508.0,1516.0,0,NaN,1,-67,-59
14,out,2,5,1639.0,1647.0,0,734.0,0,-11,-3
15,out,2,6,1758.0,1768.0,0,755.0,0,-42,-32
16,in,2,0,1800.0,1820.0,0,600.0,0,0,20
17,in,2,1,1948.0,1960.0,0,641.0,0,-2,10


In [6]:
env = EnvironmentManager()
env.start_vehicles()
env.route.load_all_pax()

In [7]:
env.step()

Vehicle 0
Scheduled time: 300, Arrival time: 202.0, Deviation: -98.0
Vehicle 0
Scheduled time: 600, Arrival time: 443.0, Deviation: -157.0
Vehicle 0
Scheduled time: 900, Arrival time: 764.0, Deviation: -136.0


([0, 2, 743.0, 71.0],
 nan,
 0,
 0,
 {'skipped_requests': 0,
  'off_schedule_trips': 0,
  'time': 835.0,
  'veh_idx': 1,
  'direction': 'out'})

In [40]:
env.step(action=0)

Vehicle 1
Scheduled time: 5700, Arrival time: 5775.0, Deviation: 75.0


([1, 1, 630.0, 75.0],
 -4.0,
 0,
 0,
 {'skipped_requests': 0,
  'off_schedule_trips': 1,
  'time': 5785.0,
  'veh_idx': 1,
  'direction': 'out'})

In [41]:
env.step(action=0)

Vehicle 1
Scheduled time: 6000, Arrival time: 6067.0, Deviation: 67.0


([0, 0, 575.0, 44.0],
 -1.0,
 0,
 0,
 {'skipped_requests': 1,
  'off_schedule_trips': 0,
  'time': 6204.0,
  'veh_idx': 0,
  'direction': 'out'})